In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/training.1600000.processed.noemoticon.csv', index_col=1)

/home/adilson/.anaconda3/envs/twitter_stock/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head()

,target,date,flag,user,tweet
id,,,,,
1467810369,0,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1467810672,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1467810917,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
1467811184,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
1467811193,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df.drop(columns=['date', 'flag', 'user'], inplace=True)

In [5]:
df.head()

,target,tweet
id,,
1467810369,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1467810672,0,is upset that he can't update his Facebook by ...
1467810917,0,@Kenichan I dived many times for the ball. Man...
1467811184,0,my whole body feels itchy and like its on fire
1467811193,0,"@nationwideclass no, it's not behaving at all...."


In [6]:
df['target'].value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [7]:
df['target'] = df['target'].apply(lambda x: True if x == 4 else False)

In [8]:
df['target'].value_counts()

True     800000
False    800000
Name: target, dtype: int64

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10, max_df=.6, ngram_range=(1, 2), sublinear_tf=True)

In [10]:
%%time
X = vectorizer.fit_transform(df['tweet'])
y = df['target']

CPU times: user 48.9 s, sys: 1.31 s, total: 50.2 s
Wall time: 50.2 s


In [11]:
X.shape

(1600000, 208130)

In [12]:
from sklearn.linear_model import RidgeClassifierCV

ridge = RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=5)

In [13]:
%%time
ridge.fit(X, y)

CPU times: user 1h 21min 48s, sys: 1h 38min 5s, total: 2h 59min 53s
Wall time: 17min 41s


RidgeClassifierCV(alphas=array([0.001, 0.01 , 0.1  , 1.   ]), cv=5)

In [14]:
print(f'Accuracy: {100*ridge.score(X, y):.2f}%')

Accuracy: 86.23%


In [15]:
import pickle

In [16]:
with open('data/model.pkl', 'wb') as f:
    pickle.dump(ridge, f)

In [17]:
with open('data/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)